# Modeling

Goal:

* Estimate an Airbnb listing's yearly revenue

Steps we will perform:

1. Preliminary modeling
2. Hyperparameter tune
3. Stack and voting regressor
4. Train on whole dataset
5. Export model

Results:

* Multiple linear regression achieves an MAE of \\$12,567.041 on the test set
* The voting regressor performs the best with an MAE of \\$11,512.009 on the test set; an **8.4%** increase in performance.

Note about performance:
* Because Airbnb doesn't release official yearly revenue data on its listings, yearly revenue had to be estimated based on the price of a listing and its availability throughout the year. This estimation in turn affects the accuracy of the model.
* Because of COVID, there is only a handful of Airbnbs that are full-time. We might want to collect our data later down the road. 
* Based on our results, it is evident that the number of beds and the location of the Airbnb are not sufficient to predict its yearly revenue. Instead, we should see if we can find data about the date the building was built (newer buildings might be more preferred than run-down buildings, amenities, and make use of the photos (nicer listings will make more).

In this notebook, we will focus our efforts on creating a model that can best estimate an Airbnb's yearly revenue.

For our evaluations, we will focus on the mean absolute error (MAE) because it is not highly sensitive to outliers like root mean squared error (RMSE) is. Since in RMSE, the errors are squared before they are averaged, RMSE gives a much higher weight to larger errors. RMSE also has a tendency to be increasingly larger than MAE as the test sample size increases. This can problematic when comparing RMSE results calculated on different sized test samples, which is frequently the case in real-world modeling.

Like housing prices, yearly revenue for listings can have many outliers, where for our model we want to create a tool that allows prospective hosts to estimate what their yearly revenue will be. This makes MAE more appropriate than RMSE. MAE is also much more interpretable and thus we will focus on MAE, but also provide other metrics as well in the final evaluation.

# Imports

In [1]:
import numpy as np
import os
import pandas as pd

from joblib import dump, load
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBRegressor

# Load data

In [2]:
fp = os.path.join('..', 'data', 'processed', 'yearly_revenue.pkl')
listings = pd.read_pickle(fp)
listings.sample(5, random_state=123)

,latitude,longitude,accommodates,bathrooms_text,bedrooms,beds,yearly_revenue,property_type_cleansed
966,47.66377,-122.35714,4,1.0,1.0,3.0,16209.0,Entire guest suite
1324,47.65701,-122.31653,5,2.0,2.0,2.0,25410.0,Entire apartment
1349,47.61214,-122.32980,4,1.0,0.0,2.0,24646.0,Entire condominium
813,47.70072,-122.27365,6,2.0,3.0,3.0,49810.0,Entire guesthouse
110,47.65915,-122.31587,3,1.0,1.0,2.0,18380.0,Entire apartment


In [3]:
# Train test split.
X = listings.drop(['yearly_revenue',], axis=1)
y = listings.yearly_revenue

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [4]:
X_train.sample(5, random_state=123)

,latitude,longitude,accommodates,bathrooms_text,bedrooms,beds,property_type_cleansed
754,47.60246,-122.29388,4,1.0,2.0,2.0,Entire apartment
152,47.68075,-122.30351,2,1.0,1.0,1.0,Private room in house
1183,47.61124,-122.33063,5,1.0,1.0,1.0,Entire condominium
1648,47.57704,-122.41478,2,1.0,1.0,1.0,Entire guesthouse
1586,47.62201,-122.29275,6,3.0,4.0,4.0,Entire house


# Preliminary modeling

The stack ensemble is the best performing model, while the random forests is the second best model. The worst model that we tried was elastic net.

In [5]:
# Create pipeline that encodes categorical variables and scales numeric variables.
numerical_features = ['latitude', 'longitude', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds']
categorical_features = X.drop(numerical_features, axis=1).columns
preprocessor = ColumnTransformer(
    transformers=[('onehot-encoder', OneHotEncoder(handle_unknown='ignore'), categorical_features),
                  ('scalar', StandardScaler(), numerical_features)])

In [6]:
cv = KFold(5, shuffle=True, random_state=123)

In [7]:
%%time
# ---------------------- Instantiate Each Model ----------------------
regressors  = {
    'linear_regressor': LinearRegression(),
    'ridge_regressor': Ridge(),
    'lasso_regressor': Lasso(),
    'elastic_regressor': ElasticNet(),
    'knn_regressor':  KNeighborsRegressor(n_neighbors=5),
    'rf_regressor': RandomForestRegressor(random_state=123),
    'xgb_regressor': XGBRegressor(objective='reg:squarederror', random_state=123)
}

pipe_dict = []


# ----- Create pipeline and evaluate preliminary performace for each model -----
pipelines = {}
metrics_df = pd.DataFrame(columns=['mean_cv'])
for i, (name, regressor) in enumerate(regressors.items()):
    pipelines[name] = Pipeline([('preprocessor', preprocessor),
                                (name, regressor)])
    
    mae = -cross_val_score(pipelines[name],
                            X_train,
                            y_train,
                            cv=cv,
                            n_jobs=-1,
                            scoring='neg_mean_absolute_error').mean()

    metrics_df = metrics_df.append(pd.DataFrame({'mean_cv': mae}, index=[name]))

# -------------------------------- Stack models --------------------------------
pipelines['stack_regressor'] = StackingRegressor(estimators=[(k, v) for k, v in pipelines.items()])

mae = -cross_val_score(pipelines['stack_regressor'],
                        X_train,
                        y_train,
                        cv=5,
                        n_jobs=-1,
                        scoring='neg_mean_absolute_error').mean()

metrics_df = metrics_df.append(pd.DataFrame({'mean_cv': mae}, index=['stack_regressor']))

CPU times: user 151 ms, sys: 102 ms, total: 254 ms
Wall time: 25.2 s


In [8]:
# Mean MAE in 5 fold CV.
metrics_df

,mean_cv
linear_regressor,11892.516130
ridge_regressor,11903.937233
lasso_regressor,11904.794185
elastic_regressor,12729.403992
knn_regressor,12060.964413
rf_regressor,11454.302699
xgb_regressor,12197.501625
stack_regressor,11329.899687


# Hyperparameter tune

Out of the tuned models, the stacked ensemble was the best with a MAE of 11133.31, followed closely by random forests and then KNN wih MAEs of 11288.79 and 11312.56 respectivley.

In [9]:
# Hyperparameter grids for models.
param_grid_ridge = {
    'ridge_regressor__alpha': [1.0, 1.5, 2.0, 2.25, 2.5, 2.75, 3, 1e5]
}

param_grid_lasso = {
    'lasso_regressor__alpha': [1, 10, 15, 20, 25, 30, 35]
}

param_grid_elastic = {
    'elastic_regressor__alpha': [0, 0.25, 0.5, 0.75, 1, 10, 15, 20, 25, 30, 35],
    'elastic_regressor__l1_ratio': [0.1, 0.25, 0.5, 0.75, 1]
}

param_grid_knn = {
    'knn_regressor__n_neighbors': [3, 5, 10, 15, 20],
    'knn_regressor__weights': ['uniform', 'distance'],
}

n_features = preprocessor.fit_transform(X_train).shape[1]
param_grid_rf = {
    'rf_regressor__n_estimators': [100, 150, 200],
    'rf_regressor__max_depth': [None, 2, 3, 4, 5],
    'rf_regressor__max_features': [np.sqrt(n_features)/n_features, 0.25, 0.5, 0.75]
}

param_grid_xgb = {
    'xgb_regressor__max_depth': [4, 5, 6],                   
    'xgb_regressor__learning_rate': [0.01, 0.1, 0.15, 0.2],
}

In [10]:
# Instantiate grid search CV for each model.
gdRidge = GridSearchCV(estimator=pipelines['ridge_regressor'],
                     param_grid=param_grid_ridge,
                     cv=cv,
                     n_jobs=-1,
                     scoring='neg_mean_absolute_error', verbose=True)

gdLasso = GridSearchCV(estimator=pipelines['lasso_regressor'],
                     param_grid=param_grid_lasso,
                     cv=cv,
                     n_jobs=-1,
                     scoring='neg_mean_absolute_error', verbose=True)

gdElastic = GridSearchCV(estimator=pipelines['elastic_regressor'],
                     param_grid=param_grid_elastic,
                     cv=cv,
                     n_jobs=-1,
                     scoring='neg_mean_absolute_error', verbose=True)

gdKNN = GridSearchCV(estimator=pipelines['knn_regressor'],
                     param_grid=param_grid_knn,
                     cv=cv,
                     n_jobs=-1,
                     scoring='neg_mean_absolute_error', verbose=True)

gdRf = GridSearchCV(estimator=pipelines['rf_regressor'],
                     param_grid=param_grid_rf,
                     cv=cv,
                     n_jobs=-1,
                     scoring='neg_mean_absolute_error', verbose=True)

gdXGB = GridSearchCV(estimator=pipelines['xgb_regressor'],
                     param_grid=param_grid_xgb,
                     cv=cv,
                     n_jobs=-1,
                     scoring='neg_mean_absolute_error', verbose=True)

In [11]:
# Grid search.
pipelines_grid = [gdRidge, gdLasso, gdElastic, gdKNN, gdRf, gdXGB]
for pipe in pipelines_grid:
    pipe.fit(X_train, y_train)
    mae = -pipe.best_score_
    name = f'tuned_{pipe.estimator.steps[1][0]}'
    pipelines[name] = pipe.best_estimator_
    metrics_df = metrics_df.append(pd.DataFrame({'mean_cv': mae}, index=[name]))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 7 candidates, totalling 35 fits
Fitting 5 folds for each of 55 candidates, totalling 275 fits


/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/pipeline.py:346: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/pipeline.py:346: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Users/nathan/.local/share/virtualenvs/airbnb_pricer_seattle-PcalCoHa/lib/python3.9/site-packages/sk

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [12]:
# Best parameters for each model.
for model in pipelines_grid:
    print(model.best_params_)

{'ridge_regressor__alpha': 2.75}
{'lasso_regressor__alpha': 15}
{'elastic_regressor__alpha': 15, 'elastic_regressor__l1_ratio': 1}
{'knn_regressor__n_neighbors': 20, 'knn_regressor__weights': 'distance'}
{'rf_regressor__max_depth': None, 'rf_regressor__max_features': 0.75, 'rf_regressor__n_estimators': 100}
{'xgb_regressor__learning_rate': 0.1, 'xgb_regressor__max_depth': 5}


In [13]:
# -------------------------------- Stack models --------------------------------
tuned_stack = StackingRegressor(estimators=[('linear', pipelines['linear_regressor']),
                                      ('ridge', gdRidge.best_estimator_),
                                      ('lasso', gdLasso.best_estimator_),
                                      ('elastic', gdElastic.best_estimator_),
                                      ('knn', gdKNN.best_estimator_),
                                      ('rf', gdRf.best_estimator_),
                                      ('xgboost', gdXGB.best_estimator_)])
# Change to RidgeCV
param_grid_stack = {
    'final_estimator': [
                        Ridge(alpha=1.6e10),
                        Ridge(alpha=1.8e10),
                        Ridge(alpha=1e11),
                        Ridge(alpha=1.2e11),
                        Ridge(alpha=1.4e11),
                        ]
}

gdStack = GridSearchCV(estimator=tuned_stack,
                      param_grid=param_grid_stack,
                      cv=cv,
                      n_jobs=-1,
                      scoring='neg_mean_absolute_error', verbose=True)

gdStack.fit(X_train, y_train)

mae = -gdStack.best_score_
pipelines['tuned_stack_regressor'] = gdStack.best_estimator_
metrics_df = metrics_df.append(pd.DataFrame({'mean_cv': mae}, index=['tuned_stack_regressor']))

Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [14]:
# Mean RMSE in 5 fold CV, with tuned models.
metrics_df

,mean_cv
linear_regressor,11892.516130
ridge_regressor,11903.937233
lasso_regressor,11904.794185
elastic_regressor,12729.403992
knn_regressor,12060.964413
rf_regressor,11454.302699
xgb_regressor,12197.501625
stack_regressor,11329.899687
tuned_ridge_regressor,11903.083480
tuned_lasso_regressor,11903.397108


# Voting regressor

The voting regressor edges past the tuned stack regressor and becomes our best performing model according to cross-validation.

In [15]:
# Find the 3 best performing models for voting regressor.
top_models = metrics_df.sort_values('mean_cv').index[:3]
metrics_df.loc[top_models]

,mean_cv
tuned_stack_regressor,11133.306893
tuned_rf_regressor,11288.787241
tuned_knn_regressor,11312.552668


In [16]:
# CV voting regressor.
estimators = [(model, pipelines[model]) for model in top_models]
pipelines['voting_regressor'] = VotingRegressor(estimators=estimators)

mae = -cross_val_score(pipelines['voting_regressor'],
                        X_train,
                        y_train,
                        cv=cv,
                        n_jobs=-1,
                        scoring='neg_mean_absolute_error').mean()

metrics_df = metrics_df.append(pd.DataFrame({'mean_cv': mae}, index=['voting_regressor']))

In [17]:
metrics_df

,mean_cv
linear_regressor,11892.516130
ridge_regressor,11903.937233
lasso_regressor,11904.794185
elastic_regressor,12729.403992
knn_regressor,12060.964413
rf_regressor,11454.302699
xgb_regressor,12197.501625
stack_regressor,11329.899687
tuned_ridge_regressor,11903.083480
tuned_lasso_regressor,11903.397108


# Evaluate best models

With a voting regressor consisting of stacked regressors of XGBoost, KNN, and
linear models, we were able to achieve an MAE of \\$11,512.01, an 8.4% increase
in performance from a multiple linear regression model. 

In [18]:
# Train and evaluate the 3 best models on the entire training dataset.
final_models = metrics_df.sort_values('mean_cv').index[:3]
final_models = final_models.append(pd.Index(['linear_regressor']))
metrics_df.loc[final_models]

,mean_cv
voting_regressor,11074.380394
tuned_stack_regressor,11133.306893
tuned_rf_regressor,11288.787241
linear_regressor,11892.516130


In [19]:
final_eval = pd.DataFrame()
for model in final_models:
    pipelines[model].fit(X_train, y_train)
    y_pred = pipelines[model].predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    final_eval = final_eval.append(pd.DataFrame({'mae': mae,
                                                 'rmse': rmse,
                                                 'mse': mse}, index=[model]))

In [20]:
final_eval.sort_values('mae')

,mae,rmse,mse
voting_regressor,11512.008638,25462.408292,6.483342e+08
tuned_stack_regressor,11566.996357,25544.313993,6.525120e+08
tuned_rf_regressor,11811.088519,25790.268806,6.651380e+08
linear_regressor,12567.040573,26323.355950,6.929191e+08


# Train on entire dataset

In [21]:
best_model_name = final_eval.sort_values('rmse').iloc[0].name
pipelines[best_model_name].fit(X, y)

VotingRegressor(estimators=[('tuned_stack_regressor',
                             StackingRegressor(estimators=[('linear',
                                                            Pipeline(steps=[('preprocessor',
                                                                             ColumnTransformer(transformers=[('onehot-encoder',
                                                                                                              OneHotEncoder(handle_unknown='ignore'),
                                                                                                              Index(['property_type_cleansed'], dtype='object')),
                                                                                                             ('scalar',
                                                                                                              StandardScaler(),
                                                                                               

# Export best model

In [22]:
fp = os.path.join('..', 'models', 'model.joblib')
dump(pipelines[best_model_name], fp)

['../models/model.joblib']